# Libraries

In [314]:
# imports
import pandas as pd
import numpy as np
import argparse

import keras
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

# Import models from another notebook
%run model.ipynb
%run util.ipynb

# Preprocessing

### Load sudoku

In [2]:
# load inputs
sudokus = pd.read_csv('./datasets/sudoku-kaggle.csv').values
print("Full shape:", sudokus.shape)
# subset = sudokus.sample(n=50000).values
# print("Subset shape:", subset.shape)

Full shape: (1000000, 2)


In [162]:
# load inputs
sudokus = pd.read_csv('./datasets/sudokus-unl.csv').values
print("Full shape:", sudokus.shape)
# subset = sudokus.sample(n=50000).values
# print("Subset shape:", subset.shape)

Full shape: (49995, 2)


In [3]:
# Split in half
used, hold = train_test_split(sudokus, test_size=0.95)
print("Subset shape:", used.shape)

Subset shape: (50000, 2)


### Split into data and labels

In [4]:
# Split into puzzles and solutions
puzzles, solutions = used[:, 0], used[:, 1]
print("Number of puzzles:", len(puzzles))
print("Number of solutions:", len(solutions))

Number of puzzles: 50000
Number of solutions: 50000


In [163]:
# Split into puzzles and solutions
puzzles, solutions = sudokus[:, 0], sudokus[:, 1]
print("Number of puzzles:", len(puzzles))
print("Number of solutions:", len(solutions))

Number of puzzles: 49995
Number of solutions: 49995


### Reshape the data

In [164]:
# Reshape to 9x9
reshape_f = lambda x: np.reshape([int(digit) for digit in x], (9, 9))
reshaped_puzzles = np.array(list(map(reshape_f, puzzles)))
reshaped_solutions = np.array(list(map(reshape_f, solutions)))
print("Shape of puzzles:", reshaped_puzzles.shape)
print("Shape of solutions:", reshaped_solutions.shape)

Shape of puzzles: (49995, 9, 9)
Shape of solutions: (49995, 9, 9)


In [238]:
# To one-hot encoding
one_hot_puzzles = to_one_hot(reshaped_puzzles)
one_hot_solutions = to_one_hot(reshaped_solutions - 1, n_classes=9)
print("Shape of puzzles:", one_hot_puzzles.shape)
print("Shape of solutions:", one_hot_solutions.shape)

Shape of puzzles: (49995, 9, 9, 10)
Shape of solutions: (49995, 9, 9, 9)


### Train test split

In [239]:
# Split into train and test set
X_train, X_test, y_train, y_test = train_test_split(one_hot_puzzles, one_hot_solutions, 
                                                    test_size=0.2, random_state=42)
print("Training data shape:", X_train.shape)
print("Training labels shape:", y_train.shape)
print("Testing data shape:", X_test.shape)
print("Testing labels shape:", y_test.shape)

Training data shape: (39996, 9, 9, 10)
Training labels shape: (39996, 9, 9, 9)
Testing data shape: (9999, 9, 9, 10)
Testing labels shape: (9999, 9, 9, 9)


In [240]:
# validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
print("Training data shape:", X_train.shape)
print("Training labels shape:", y_train.shape)
print("Validating data shape:", X_val.shape)
print("Validating labels shape:", y_val.shape)

Training data shape: (31996, 9, 9, 10)
Training labels shape: (31996, 9, 9, 9)
Validating data shape: (8000, 9, 9, 10)
Validating labels shape: (8000, 9, 9, 9)


In [241]:
# Create new X_train from y_train
X_train_completed = to_puzzles(y_train)
X_val_completed = to_puzzles(y_val)
print("Training data shape:", X_train_completed.shape)
print("Training labels shape:", y_train.shape)
print("Validating data shape:", X_val_completed.shape)
print("Validating labels shape:", y_val.shape)

Training data shape: (31996, 9, 9, 10)
Training labels shape: (31996, 9, 9, 9)
Validating data shape: (8000, 9, 9, 10)
Validating labels shape: (8000, 9, 9, 9)


# Start training

In [169]:
# initialize model
model = get_model(X_train.shape[1:], model_id='dense_model')
print("Model input shape:", model.input.shape)
print("Model output shape:", len(model.output))

Model input shape: (?, 9, 9, 10)
Model output shape: 81


In [172]:
%%time
early_stop = EarlyStopping(patience=2, verbose=1)

i = 1
for nb_epochs, nb_delete in zip(
        [1, 2, 3, 4, 6, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],  # epochs for each round
        [1, 2, 3, 4, 6, 8, 10, 12, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70]  # digit to pull off
):
    print('Pass n° {} ...'.format(i))
    i += 1
    
    model.fit(
        delete_digits(X_train_completed, nb_delete),  # delete digits from training sample
        [y_train[:, i, j, :] for i in range(9) for j in range(9)],
        validation_data=(
            delete_digits(X_val_completed, nb_delete), # delete same amount of digit from validation sample
            [y_val[:, i, j, :] for i in range(9) for j in range(9)]),
        batch_size=256,
        epochs=nb_epochs,
        verbose=1,
        callbacks=[early_stop]
    )
    keras.models.save_model(model, "./models/unl-model-%d.h5" % nb_delete)

Pass n° 1 ...
Train on 31996 samples, validate on 8000 samples
Epoch 1/1
31996/31996 [==============================] - 66s 2ms/step - loss: 1.8251 - dense_165_loss: 0.0221 - dense_166_loss: 0.0226 - dense_167_loss: 0.0227 - dense_168_loss: 0.0222 - dense_169_loss: 0.0225 - dense_170_loss: 0.0220 - dense_171_loss: 0.0208 - dense_172_loss: 0.0243 - dense_173_loss: 0.0225 - dense_174_loss: 0.0248 - dense_175_loss: 0.0219 - dense_176_loss: 0.0233 - dense_177_loss: 0.0216 - dense_178_loss: 0.0236 - dense_179_loss: 0.0244 - dense_180_loss: 0.0221 - dense_181_loss: 0.0221 - dense_182_loss: 0.0218 - dense_183_loss: 0.0226 - dense_184_loss: 0.0227 - dense_185_loss: 0.0228 - dense_186_loss: 0.0214 - dense_187_loss: 0.0238 - dense_188_loss: 0.0232 - dense_189_loss: 0.0230 - dense_190_loss: 0.0224 - dense_191_loss: 0.0238 - dense_192_loss: 0.0210 - dense_193_loss: 0.0222 - dense_194_loss: 0.0234 - dense_195_loss: 0.0225 - dense_196_loss: 0.0222 - dense_197_loss: 0.0218 - dense_198_loss: 0.0229 - 

Pass n° 2 ...
Train on 31996 samples, validate on 8000 samples
Epoch 1/2
31996/31996 [==============================] - 62s 2ms/step - loss: 1.6520 - dense_165_loss: 0.0200 - dense_166_loss: 0.0191 - dense_167_loss: 0.0197 - dense_168_loss: 0.0186 - dense_169_loss: 0.0216 - dense_170_loss: 0.0199 - dense_171_loss: 0.0186 - dense_172_loss: 0.0208 - dense_173_loss: 0.0213 - dense_174_loss: 0.0205 - dense_175_loss: 0.0194 - dense_176_loss: 0.0204 - dense_177_loss: 0.0207 - dense_178_loss: 0.0207 - dense_179_loss: 0.0188 - dense_180_loss: 0.0211 - dense_181_loss: 0.0228 - dense_182_loss: 0.0195 - dense_183_loss: 0.0186 - dense_184_loss: 0.0208 - dense_185_loss: 0.0202 - dense_186_loss: 0.0211 - dense_187_loss: 0.0204 - dense_188_loss: 0.0195 - dense_189_loss: 0.0204 - dense_190_loss: 0.0200 - dense_191_loss: 0.0207 - dense_192_loss: 0.0203 - dense_193_loss: 0.0179 - dense_194_loss: 0.0210 - dense_195_loss: 0.0203 - dense_196_loss: 0.0200 - dense_197_loss: 0.0197 - dense_198_loss: 0.0218 - 

Epoch 2/2
31996/31996 [==============================] - 66s 2ms/step - loss: 1.0911 - dense_165_loss: 0.0134 - dense_166_loss: 0.0138 - dense_167_loss: 0.0135 - dense_168_loss: 0.0123 - dense_169_loss: 0.0127 - dense_170_loss: 0.0130 - dense_171_loss: 0.0126 - dense_172_loss: 0.0126 - dense_173_loss: 0.0126 - dense_174_loss: 0.0117 - dense_175_loss: 0.0143 - dense_176_loss: 0.0133 - dense_177_loss: 0.0158 - dense_178_loss: 0.0152 - dense_179_loss: 0.0132 - dense_180_loss: 0.0140 - dense_181_loss: 0.0150 - dense_182_loss: 0.0139 - dense_183_loss: 0.0154 - dense_184_loss: 0.0134 - dense_185_loss: 0.0130 - dense_186_loss: 0.0144 - dense_187_loss: 0.0116 - dense_188_loss: 0.0142 - dense_189_loss: 0.0145 - dense_190_loss: 0.0135 - dense_191_loss: 0.0138 - dense_192_loss: 0.0124 - dense_193_loss: 0.0136 - dense_194_loss: 0.0146 - dense_195_loss: 0.0125 - dense_196_loss: 0.0130 - dense_197_loss: 0.0129 - dense_198_loss: 0.0130 - dense_199_loss: 0.0145 - dense_200_loss: 0.0142 - dense_201_los

Pass n° 3 ...
Train on 31996 samples, validate on 8000 samples
Epoch 1/3
31996/31996 [==============================] - 65s 2ms/step - loss: 1.6538 - dense_165_loss: 0.0223 - dense_166_loss: 0.0205 - dense_167_loss: 0.0208 - dense_168_loss: 0.0200 - dense_169_loss: 0.0195 - dense_170_loss: 0.0223 - dense_171_loss: 0.0204 - dense_172_loss: 0.0203 - dense_173_loss: 0.0220 - dense_174_loss: 0.0210 - dense_175_loss: 0.0207 - dense_176_loss: 0.0204 - dense_177_loss: 0.0200 - dense_178_loss: 0.0219 - dense_179_loss: 0.0201 - dense_180_loss: 0.0200 - dense_181_loss: 0.0189 - dense_182_loss: 0.0233 - dense_183_loss: 0.0201 - dense_184_loss: 0.0232 - dense_185_loss: 0.0223 - dense_186_loss: 0.0197 - dense_187_loss: 0.0191 - dense_188_loss: 0.0203 - dense_189_loss: 0.0200 - dense_190_loss: 0.0199 - dense_191_loss: 0.0234 - dense_192_loss: 0.0196 - dense_193_loss: 0.0207 - dense_194_loss: 0.0208 - dense_195_loss: 0.0201 - dense_196_loss: 0.0198 - dense_197_loss: 0.0182 - dense_198_loss: 0.0213 - 

Epoch 2/3
31996/31996 [==============================] - 62s 2ms/step - loss: 1.1978 - dense_165_loss: 0.0156 - dense_166_loss: 0.0138 - dense_167_loss: 0.0171 - dense_168_loss: 0.0167 - dense_169_loss: 0.0146 - dense_170_loss: 0.0147 - dense_171_loss: 0.0140 - dense_172_loss: 0.0151 - dense_173_loss: 0.0155 - dense_174_loss: 0.0166 - dense_175_loss: 0.0161 - dense_176_loss: 0.0159 - dense_177_loss: 0.0139 - dense_178_loss: 0.0130 - dense_179_loss: 0.0162 - dense_180_loss: 0.0139 - dense_181_loss: 0.0132 - dense_182_loss: 0.0164 - dense_183_loss: 0.0174 - dense_184_loss: 0.0158 - dense_185_loss: 0.0156 - dense_186_loss: 0.0144 - dense_187_loss: 0.0143 - dense_188_loss: 0.0139 - dense_189_loss: 0.0134 - dense_190_loss: 0.0140 - dense_191_loss: 0.0167 - dense_192_loss: 0.0141 - dense_193_loss: 0.0146 - dense_194_loss: 0.0139 - dense_195_loss: 0.0133 - dense_196_loss: 0.0144 - dense_197_loss: 0.0116 - dense_198_loss: 0.0142 - dense_199_loss: 0.0139 - dense_200_loss: 0.0159 - dense_201_los

...

Epoch 3/10
31996/31996 [==============================] - 62s 2ms/step - loss: 39.9103 - dense_165_loss: 0.4867 - dense_166_loss: 0.4892 - dense_167_loss: 0.4918 - dense_168_loss: 0.4902 - dense_169_loss: 0.4866 - dense_170_loss: 0.4853 - dense_171_loss: 0.4883 - dense_172_loss: 0.4924 - dense_173_loss: 0.4936 - dense_174_loss: 0.4879 - dense_175_loss: 0.4964 - dense_176_loss: 0.4928 - dense_177_loss: 0.4864 - dense_178_loss: 0.4917 - dense_179_loss: 0.4903 - dense_180_loss: 0.4981 - dense_181_loss: 0.4907 - dense_182_loss: 0.4877 - dense_183_loss: 0.4917 - dense_184_loss: 0.4957 - dense_185_loss: 0.4926 - dense_186_loss: 0.4951 - dense_187_loss: 0.4965 - dense_188_loss: 0.4906 - dense_189_loss: 0.4890 - dense_190_loss: 0.4923 - dense_191_loss: 0.4910 - dense_192_loss: 0.4913 - dense_193_loss: 0.4996 - dense_194_loss: 0.4888 - dense_195_loss: 0.5002 - dense_196_loss: 0.4934 - dense_197_loss: 0.4857 - dense_198_loss: 0.4901 - dense_199_loss: 0.4924 - dense_200_loss: 0.4866 - dense_201_l

Epoch 00003: early stopping
Pass n° 20 ...
Train on 31996 samples, validate on 8000 samples
Epoch 1/10
31996/31996 [==============================] - 64s 2ms/step - loss: 50.1103 - dense_165_loss: 0.6156 - dense_166_loss: 0.6138 - dense_167_loss: 0.6161 - dense_168_loss: 0.6150 - dense_169_loss: 0.6127 - dense_170_loss: 0.6158 - dense_171_loss: 0.6132 - dense_172_loss: 0.6184 - dense_173_loss: 0.6191 - dense_174_loss: 0.6208 - dense_175_loss: 0.6194 - dense_176_loss: 0.6080 - dense_177_loss: 0.6222 - dense_178_loss: 0.6184 - dense_179_loss: 0.6133 - dense_180_loss: 0.6185 - dense_181_loss: 0.6192 - dense_182_loss: 0.6191 - dense_183_loss: 0.6201 - dense_184_loss: 0.6139 - dense_185_loss: 0.6082 - dense_186_loss: 0.6204 - dense_187_loss: 0.6174 - dense_188_loss: 0.6128 - dense_189_loss: 0.6246 - dense_190_loss: 0.6154 - dense_191_loss: 0.6154 - dense_192_loss: 0.6065 - dense_193_loss: 0.6190 - dense_194_loss: 0.6184 - dense_195_loss: 0.6180 - dense_196_loss: 0.6152 - dense_197_loss: 0.6

Epoch 2/10
31996/31996 [==============================] - 62s 2ms/step - loss: 46.7813 - dense_165_loss: 0.5741 - dense_166_loss: 0.5719 - dense_167_loss: 0.5740 - dense_168_loss: 0.5763 - dense_169_loss: 0.5706 - dense_170_loss: 0.5739 - dense_171_loss: 0.5735 - dense_172_loss: 0.5741 - dense_173_loss: 0.5775 - dense_174_loss: 0.5748 - dense_175_loss: 0.5823 - dense_176_loss: 0.5670 - dense_177_loss: 0.5820 - dense_178_loss: 0.5795 - dense_179_loss: 0.5737 - dense_180_loss: 0.5764 - dense_181_loss: 0.5766 - dense_182_loss: 0.5766 - dense_183_loss: 0.5827 - dense_184_loss: 0.5711 - dense_185_loss: 0.5683 - dense_186_loss: 0.5798 - dense_187_loss: 0.5750 - dense_188_loss: 0.5736 - dense_189_loss: 0.5836 - dense_190_loss: 0.5718 - dense_191_loss: 0.5746 - dense_192_loss: 0.5670 - dense_193_loss: 0.5784 - dense_194_loss: 0.5752 - dense_195_loss: 0.5743 - dense_196_loss: 0.5780 - dense_197_loss: 0.5841 - dense_198_loss: 0.5817 - dense_199_loss: 0.5780 - dense_200_loss: 0.5767 - dense_201_l

Epoch 3/10
31996/31996 [==============================] - 62s 2ms/step - loss: 45.3879 - dense_165_loss: 0.5567 - dense_166_loss: 0.5569 - dense_167_loss: 0.5589 - dense_168_loss: 0.5581 - dense_169_loss: 0.5542 - dense_170_loss: 0.5538 - dense_171_loss: 0.5570 - dense_172_loss: 0.5592 - dense_173_loss: 0.5600 - dense_174_loss: 0.5613 - dense_175_loss: 0.5598 - dense_176_loss: 0.5525 - dense_177_loss: 0.5632 - dense_178_loss: 0.5593 - dense_179_loss: 0.5559 - dense_180_loss: 0.5589 - dense_181_loss: 0.5569 - dense_182_loss: 0.5609 - dense_183_loss: 0.5678 - dense_184_loss: 0.5552 - dense_185_loss: 0.5490 - dense_186_loss: 0.5629 - dense_187_loss: 0.5578 - dense_188_loss: 0.5560 - dense_189_loss: 0.5670 - dense_190_loss: 0.5581 - dense_191_loss: 0.5550 - dense_192_loss: 0.5521 - dense_193_loss: 0.5588 - dense_194_loss: 0.5586 - dense_195_loss: 0.5560 - dense_196_loss: 0.5570 - dense_197_loss: 0.5626 - dense_198_loss: 0.5615 - dense_199_loss: 0.5592 - dense_200_loss: 0.5615 - dense_201_l

Epoch 00003: early stopping
Wall time: 1h 9min 46s


In [10]:
# Split into puzzles and solutions
puzzles_t, solutions_t = hold[:, 0], hold[:, 1]
print("Number of puzzles:", len(puzzles_t))
print("Number of solutions:", len(solutions_t))

# Reshape to 9x9
reshaped_puzzles_t = np.array(list(map(reshape_f, puzzles_t)))
reshaped_solutions_t = np.array(list(map(reshape_f, solutions_t)))
print("Shape of puzzles:", reshaped_puzzles_t.shape)
print("Shape of solutions:", reshaped_solutions_t.shape)

# To one-hot encoding
one_hot_puzzles_t = to_one_hot(reshaped_puzzles_t)
one_hot_solutions_t = to_one_hot(reshaped_solutions_t - 1)
print("Shape of puzzles:", one_hot_puzzles_t.shape)
print("Shape of solutions:", one_hot_solutions_t.shape)

Number of puzzles: 950000
Number of solutions: 950000
Shape of puzzles: (950000, 9, 9)
Shape of solutions: (950000, 9, 9)
Shape of puzzles: (950000, 9, 9, 10)
Shape of solutions: (950000, 9, 9, 9)


### Convolutional Layers

In [24]:
# initialize model
c_model = get_model(X_train.shape[1:], model_id='conv_model')
print("Model input shape:", c_model.input.shape)
print("Model output shape:", len(c_model.output))

Model input shape: (?, 9, 9, 10)
Model output shape: 81


In [27]:
%%time
# train model
c_model.fit(delete_digits(X_train_completed, 0), 
            [y_train[:, i, j, :] for i in range(9) for j in range(9)], 
            batch_size=128, 
            epochs=1, 
            verbose=1)
keras.models.save_model(c_model, "./models/conv_model-0.h5")

Epoch 1/1
40500/40500 [==============================] - 91s 2ms/step - loss: 48.2136 - dense_83_loss: 0.8554 - dense_84_loss: 0.6764 - dense_85_loss: 0.6912 - dense_86_loss: 0.7023 - dense_87_loss: 0.6883 - dense_88_loss: 0.6906 - dense_89_loss: 0.6770 - dense_90_loss: 0.6611 - dense_91_loss: 0.8915 - dense_92_loss: 0.6684 - dense_93_loss: 0.5002 - dense_94_loss: 0.5021 - dense_95_loss: 0.5245 - dense_96_loss: 0.5048 - dense_97_loss: 0.5083 - dense_98_loss: 0.5164 - dense_99_loss: 0.5074 - dense_100_loss: 0.6750 - dense_101_loss: 0.7086 - dense_102_loss: 0.5109 - dense_103_loss: 0.5216 - dense_104_loss: 0.5300 - dense_105_loss: 0.5334 - dense_106_loss: 0.5107 - dense_107_loss: 0.5209 - dense_108_loss: 0.5197 - dense_109_loss: 0.7111 - dense_110_loss: 0.6915 - dense_111_loss: 0.5157 - dense_112_loss: 0.5172 - dense_113_loss: 0.5300 - dense_114_loss: 0.5313 - dense_115_loss: 0.5182 - dense_116_loss: 0.5188 - dense_117_loss: 0.5142 - dense_118_loss: 0.7131 - dense_119_loss: 0.6877 - dens

In [35]:
%%time
early_stop = EarlyStopping(patience=2, verbose=1)

i = 1
for nb_epochs, nb_delete in zip(
        [1, 2, 3, 4, 6, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],  # epochs for each round
        [1, 2, 3, 4, 6, 8, 10, 12, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70]  # digit to pull off
):
    print('Pass n° {} ...'.format(i))
    i += 1
    
    c_model.fit(
        delete_digits(X_train_completed, nb_delete),  # delete digits from training sample
        [y_train[:, i, j, :] for i in range(9) for j in range(9)],
        validation_data=(
            delete_digits(X_val_completed, nb_delete), # delete same amount of digit from validation sample
            [y_val[:, i, j, :] for i in range(9) for j in range(9)]),
        batch_size=128,
        epochs=nb_epochs,
        verbose=1,
        callbacks=[early_stop]
    )
    keras.models.save_model(c_model, "./models/conv_model-%d.h5" % nb_delete)

Pass n° 1 ...
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 88s 2ms/step - loss: 6.1046 - dense_83_loss: 0.1267 - dense_84_loss: 0.0747 - dense_85_loss: 0.0917 - dense_86_loss: 0.0864 - dense_87_loss: 0.0821 - dense_88_loss: 0.0930 - dense_89_loss: 0.0881 - dense_90_loss: 0.0717 - dense_91_loss: 0.1311 - dense_92_loss: 0.0810 - dense_93_loss: 0.0538 - dense_94_loss: 0.0594 - dense_95_loss: 0.0661 - dense_96_loss: 0.0598 - dense_97_loss: 0.0657 - dense_98_loss: 0.0590 - dense_99_loss: 0.0544 - dense_100_loss: 0.0772 - dense_101_loss: 0.0913 - dense_102_loss: 0.0588 - dense_103_loss: 0.0708 - dense_104_loss: 0.0679 - dense_105_loss: 0.0665 - dense_106_loss: 0.0648 - dense_107_loss: 0.0719 - dense_108_loss: 0.0588 - dense_109_loss: 0.0892 - dense_110_loss: 0.0931 - dense_111_loss: 0.0612 - dense_112_loss: 0.0646 - dense_113_loss: 0.0678 - dense_114_loss: 0.0689 - dense_115_loss: 0.0720 - dense_116_loss: 0.0644 - dense_117_loss: 0

Pass n° 2 ...
Train on 40500 samples, validate on 4500 samples
Epoch 1/2
40500/40500 [==============================] - 86s 2ms/step - loss: 6.3582 - dense_83_loss: 0.0857 - dense_84_loss: 0.0689 - dense_85_loss: 0.0808 - dense_86_loss: 0.0807 - dense_87_loss: 0.0810 - dense_88_loss: 0.0815 - dense_89_loss: 0.0799 - dense_90_loss: 0.0725 - dense_91_loss: 0.0873 - dense_92_loss: 0.0734 - dense_93_loss: 0.0740 - dense_94_loss: 0.0779 - dense_95_loss: 0.0714 - dense_96_loss: 0.0658 - dense_97_loss: 0.0691 - dense_98_loss: 0.0760 - dense_99_loss: 0.0675 - dense_100_loss: 0.0712 - dense_101_loss: 0.0825 - dense_102_loss: 0.0796 - dense_103_loss: 0.0801 - dense_104_loss: 0.0785 - dense_105_loss: 0.0803 - dense_106_loss: 0.0850 - dense_107_loss: 0.0814 - dense_108_loss: 0.0810 - dense_109_loss: 0.0848 - dense_110_loss: 0.0828 - dense_111_loss: 0.0804 - dense_112_loss: 0.0883 - dense_113_loss: 0.0811 - dense_114_loss: 0.0798 - dense_115_loss: 0.0844 - dense_116_loss: 0.0794 - dense_117_loss: 0

Epoch 2/2
40500/40500 [==============================] - 86s 2ms/step - loss: 4.7298 - dense_83_loss: 0.0592 - dense_84_loss: 0.0567 - dense_85_loss: 0.0579 - dense_86_loss: 0.0564 - dense_87_loss: 0.0610 - dense_88_loss: 0.0588 - dense_89_loss: 0.0627 - dense_90_loss: 0.0523 - dense_91_loss: 0.0631 - dense_92_loss: 0.0521 - dense_93_loss: 0.0517 - dense_94_loss: 0.0588 - dense_95_loss: 0.0567 - dense_96_loss: 0.0506 - dense_97_loss: 0.0550 - dense_98_loss: 0.0588 - dense_99_loss: 0.0543 - dense_100_loss: 0.0535 - dense_101_loss: 0.0566 - dense_102_loss: 0.0621 - dense_103_loss: 0.0593 - dense_104_loss: 0.0608 - dense_105_loss: 0.0584 - dense_106_loss: 0.0582 - dense_107_loss: 0.0622 - dense_108_loss: 0.0598 - dense_109_loss: 0.0664 - dense_110_loss: 0.0654 - dense_111_loss: 0.0585 - dense_112_loss: 0.0650 - dense_113_loss: 0.0578 - dense_114_loss: 0.0593 - dense_115_loss: 0.0607 - dense_116_loss: 0.0536 - dense_117_loss: 0.0555 - dense_118_loss: 0.0564 - dense_119_loss: 0.0583 - dense

Pass n° 3 ...
Train on 40500 samples, validate on 4500 samples
Epoch 1/3
40500/40500 [==============================] - 85s 2ms/step - loss: 6.0113 - dense_83_loss: 0.0623 - dense_84_loss: 0.0686 - dense_85_loss: 0.0725 - dense_86_loss: 0.0775 - dense_87_loss: 0.0694 - dense_88_loss: 0.0716 - dense_89_loss: 0.0772 - dense_90_loss: 0.0666 - dense_91_loss: 0.0661 - dense_92_loss: 0.0620 - dense_93_loss: 0.0734 - dense_94_loss: 0.0763 - dense_95_loss: 0.0776 - dense_96_loss: 0.0753 - dense_97_loss: 0.0777 - dense_98_loss: 0.0719 - dense_99_loss: 0.0734 - dense_100_loss: 0.0637 - dense_101_loss: 0.0741 - dense_102_loss: 0.0751 - dense_103_loss: 0.0822 - dense_104_loss: 0.0804 - dense_105_loss: 0.0785 - dense_106_loss: 0.0794 - dense_107_loss: 0.0823 - dense_108_loss: 0.0794 - dense_109_loss: 0.0718 - dense_110_loss: 0.0716 - dense_111_loss: 0.0786 - dense_112_loss: 0.0762 - dense_113_loss: 0.0773 - dense_114_loss: 0.0745 - dense_115_loss: 0.0740 - dense_116_loss: 0.0761 - dense_117_loss: 0

Epoch 2/3
40500/40500 [==============================] - 85s 2ms/step - loss: 4.2187 - dense_83_loss: 0.0491 - dense_84_loss: 0.0467 - dense_85_loss: 0.0501 - dense_86_loss: 0.0554 - dense_87_loss: 0.0488 - dense_88_loss: 0.0503 - dense_89_loss: 0.0502 - dense_90_loss: 0.0470 - dense_91_loss: 0.0508 - dense_92_loss: 0.0439 - dense_93_loss: 0.0484 - dense_94_loss: 0.0518 - dense_95_loss: 0.0523 - dense_96_loss: 0.0497 - dense_97_loss: 0.0533 - dense_98_loss: 0.0557 - dense_99_loss: 0.0495 - dense_100_loss: 0.0459 - dense_101_loss: 0.0520 - dense_102_loss: 0.0518 - dense_103_loss: 0.0560 - dense_104_loss: 0.0484 - dense_105_loss: 0.0561 - dense_106_loss: 0.0518 - dense_107_loss: 0.0574 - dense_108_loss: 0.0544 - dense_109_loss: 0.0489 - dense_110_loss: 0.0495 - dense_111_loss: 0.0514 - dense_112_loss: 0.0538 - dense_113_loss: 0.0493 - dense_114_loss: 0.0541 - dense_115_loss: 0.0562 - dense_116_loss: 0.0538 - dense_117_loss: 0.0542 - dense_118_loss: 0.0517 - dense_119_loss: 0.0505 - dense

...

Epoch 3/10
40500/40500 [==============================] - 83s 2ms/step - loss: 23.5828 - dense_83_loss: 0.2854 - dense_84_loss: 0.2902 - dense_85_loss: 0.2932 - dense_86_loss: 0.2962 - dense_87_loss: 0.2864 - dense_88_loss: 0.2919 - dense_89_loss: 0.2864 - dense_90_loss: 0.2851 - dense_91_loss: 0.2906 - dense_92_loss: 0.2872 - dense_93_loss: 0.2895 - dense_94_loss: 0.2903 - dense_95_loss: 0.2865 - dense_96_loss: 0.2864 - dense_97_loss: 0.2916 - dense_98_loss: 0.2940 - dense_99_loss: 0.2849 - dense_100_loss: 0.2859 - dense_101_loss: 0.2881 - dense_102_loss: 0.2880 - dense_103_loss: 0.2912 - dense_104_loss: 0.2922 - dense_105_loss: 0.2957 - dense_106_loss: 0.2909 - dense_107_loss: 0.2854 - dense_108_loss: 0.2938 - dense_109_loss: 0.2880 - dense_110_loss: 0.2857 - dense_111_loss: 0.2950 - dense_112_loss: 0.2900 - dense_113_loss: 0.2948 - dense_114_loss: 0.2949 - dense_115_loss: 0.2958 - dense_116_loss: 0.2902 - dense_117_loss: 0.2914 - dense_118_loss: 0.2887 - dense_119_loss: 0.2885 - den

Epoch 00003: early stopping
Pass n° 17 ...
Train on 40500 samples, validate on 4500 samples
Epoch 1/10
40500/40500 [==============================] - 82s 2ms/step - loss: 32.4503 - dense_83_loss: 0.3933 - dense_84_loss: 0.4000 - dense_85_loss: 0.4044 - dense_86_loss: 0.3972 - dense_87_loss: 0.4027 - dense_88_loss: 0.3983 - dense_89_loss: 0.3976 - dense_90_loss: 0.4018 - dense_91_loss: 0.3981 - dense_92_loss: 0.3954 - dense_93_loss: 0.4017 - dense_94_loss: 0.4029 - dense_95_loss: 0.3997 - dense_96_loss: 0.4002 - dense_97_loss: 0.4015 - dense_98_loss: 0.3950 - dense_99_loss: 0.3940 - dense_100_loss: 0.3995 - dense_101_loss: 0.4025 - dense_102_loss: 0.4010 - dense_103_loss: 0.3972 - dense_104_loss: 0.3997 - dense_105_loss: 0.4024 - dense_106_loss: 0.3967 - dense_107_loss: 0.3957 - dense_108_loss: 0.4009 - dense_109_loss: 0.3960 - dense_110_loss: 0.3979 - dense_111_loss: 0.4031 - dense_112_loss: 0.4023 - dense_113_loss: 0.4004 - dense_114_loss: 0.4038 - dense_115_loss: 0.4007 - dense_116_l

Epoch 2/10
40500/40500 [==============================] - 83s 2ms/step - loss: 29.6137 - dense_83_loss: 0.3597 - dense_84_loss: 0.3678 - dense_85_loss: 0.3688 - dense_86_loss: 0.3624 - dense_87_loss: 0.3638 - dense_88_loss: 0.3678 - dense_89_loss: 0.3605 - dense_90_loss: 0.3660 - dense_91_loss: 0.3636 - dense_92_loss: 0.3594 - dense_93_loss: 0.3663 - dense_94_loss: 0.3704 - dense_95_loss: 0.3645 - dense_96_loss: 0.3637 - dense_97_loss: 0.3657 - dense_98_loss: 0.3595 - dense_99_loss: 0.3618 - dense_100_loss: 0.3622 - dense_101_loss: 0.3674 - dense_102_loss: 0.3664 - dense_103_loss: 0.3617 - dense_104_loss: 0.3670 - dense_105_loss: 0.3688 - dense_106_loss: 0.3602 - dense_107_loss: 0.3619 - dense_108_loss: 0.3617 - dense_109_loss: 0.3614 - dense_110_loss: 0.3640 - dense_111_loss: 0.3667 - dense_112_loss: 0.3651 - dense_113_loss: 0.3658 - dense_114_loss: 0.3695 - dense_115_loss: 0.3678 - dense_116_loss: 0.3634 - dense_117_loss: 0.3657 - dense_118_loss: 0.3644 - dense_119_loss: 0.3671 - den

Epoch 3/10
40500/40500 [==============================] - 82s 2ms/step - loss: 28.4114 - dense_83_loss: 0.3445 - dense_84_loss: 0.3507 - dense_85_loss: 0.3579 - dense_86_loss: 0.3508 - dense_87_loss: 0.3527 - dense_88_loss: 0.3503 - dense_89_loss: 0.3443 - dense_90_loss: 0.3508 - dense_91_loss: 0.3510 - dense_92_loss: 0.3471 - dense_93_loss: 0.3549 - dense_94_loss: 0.3571 - dense_95_loss: 0.3492 - dense_96_loss: 0.3490 - dense_97_loss: 0.3514 - dense_98_loss: 0.3464 - dense_99_loss: 0.3470 - dense_100_loss: 0.3463 - dense_101_loss: 0.3546 - dense_102_loss: 0.3502 - dense_103_loss: 0.3479 - dense_104_loss: 0.3498 - dense_105_loss: 0.3551 - dense_106_loss: 0.3466 - dense_107_loss: 0.3480 - dense_108_loss: 0.3495 - dense_109_loss: 0.3472 - dense_110_loss: 0.3491 - dense_111_loss: 0.3526 - dense_112_loss: 0.3498 - dense_113_loss: 0.3501 - dense_114_loss: 0.3549 - dense_115_loss: 0.3489 - dense_116_loss: 0.3447 - dense_117_loss: 0.3502 - dense_118_loss: 0.3496 - dense_119_loss: 0.3499 - den

Epoch 00003: early stopping
Wall time: 1h 15min 4s


### Given 17 cells

In [131]:
given17 = pd.read_csv('./datasets/17-givens.csv').values
given17.shape

(49151, 1)

In [53]:
# Reshape to 9x9
given17 = np.array(list(map(reshape_f, given17[:, 0])))
print("Shape of puzzles:", given17.shape)

# To one-hot encoding
given17 = np.array(to_one_hot(given17))
print("Shape of puzzles:", given17.shape)

Shape of puzzles: (49151, 9, 9)
Shape of puzzles: (49151, 9, 9, 10)


### HardestSudokusThread

In [132]:
hst = pd.read_csv('./datasets/HardestSudokusThread.csv').values
hst.shape

(375, 1)

In [82]:
# Reshape to 9x9
hst = np.array(list(map(reshape_f, hst[:, 0])))
print("Shape of puzzles:", hst.shape)

# To one-hot encoding
hst = np.array(to_one_hot(hst))
print("Shape of puzzles:", hst.shape)

Shape of puzzles: (375, 9, 9)
Shape of puzzles: (375, 9, 9, 10)


### UNL Sudokus

In [90]:
import json

In [133]:
with open('./datasets/sudokus-unl.json') as read_file:
    sudokus_unl = json.load(read_file)

sudokus_unl = np.array(sudokus_unl)
sudokus_unl.shape

(472, 13)

In [94]:
sudokus_unl[0]

array(['99', 'Base', 'Omaha World-Herald', '1', '2', '5',
       'March 24, 2009', '', 'Puzzle added by Jason Gaare',
       '100000026200709010030060000000800700710526038003007000000010050050603007960000004',
       '1', 'GAC', '29'], dtype='<U81')

In [95]:
# Reshape to 9x9
sudokus_unl = np.array(list(map(reshape_f, sudokus_unl[:, 9])))
print("Shape of puzzles:", sudokus_unl.shape)

# To one-hot encoding
sudokus_unl = np.array(to_one_hot(sudokus_unl))
print("Shape of puzzles:", sudokus_unl.shape)

Shape of puzzles: (472, 9, 9)
Shape of puzzles: (472, 9, 9, 10)


In [109]:
results = np.expand_dims(np.array(deltas_t), axis=1)
results

array([[False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [

In [110]:
full_unl = np.concatenate((sudokus_unl, results), axis=1)
full_unl.shape

(472, 14)

In [127]:
solvable = full_unl[np.where(full_unl[:, -1] == 'True')]
np.where(full_unl[:, -1] == 'True')

(array([  1,   6,  12,  14,  28,  36,  48,  49,  50,  53,  55,  66,  73,
         76,  80,  84,  86,  87, 467], dtype=int64),)

In [128]:
# algorithm
solvable[:, -3]

array(['AC', 'GAC', 'GAC', 'GAC', 'SAC', 'AC', 'AC', 'AC', 'AC', 'AC',
       'AC', 'SAC', 'AC', 'Unsolved', 'AC', 'AC', 'AC', 'AC', 'GAC'],
      dtype='<U81')

In [130]:
# difficulty
solvable[:, 4]

array(['2', '1', '5', '5', '4', '3', '4', '1', '1', '1', '5', '4', '1',
       '1', '1', '1', '1', '1', '2'], dtype='<U81')

In [125]:
solvable[np.where(solvable[:, -3] == 'Unsolved')]

array([['161', 'Base', '', '1', '1', '5', '2-Sol', '2-Sol', '',
        '906070403000400200070023010500000100040208060003000005030700050007005000405010708',
        '2', 'Unsolved', '29', 'True']], dtype='<U81')

### Prepare new dataset

In [139]:
unl = np.concatenate((given17, hst, sudokus_unl), axis=0)
unl.shape

(49998, 1)

In [145]:
unl_sols = solve(unl)
unl_sols.shape

(49998, 1)

In [155]:
unl_dataset = np.concatenate((unl, unl_sols), axis=1)
unl_dataset.shape

(49998, 2)

In [159]:
unl_dataset = unl_dataset[unl_dataset[:, 1] != None]
unl_dataset.shape

(49995, 2)

In [161]:
write_csv(['puzzle', 'solution'], unl_dataset, "./datasets/sudokus-unl.csv")